## Cyberbullying Auto Detection
*Wenqu Wang, Casey Yoon*

### Import Packages

In [1]:
# NumPy, TensorFlow, os
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow import keras
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection, naive_bayes, svm
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer

In [2]:
os.listdir()

['twitter_sentiment.ipynb',
 'twitter_racism_parsed_dataset.csv',
 '.git',
 'twitter_parsed_dataset.csv',
 'Untitled.ipynb',
 '.ipynb_checkpoints',
 'twitter_sexism_parsed_dataset.csv']

In [3]:
### We aggregate all the data into one dataframe

parsed = pd.read_csv('twitter_parsed_dataset.csv')
racism = pd.read_csv('twitter_racism_parsed_dataset.csv')
sexism = pd.read_csv('twitter_sexism_parsed_dataset.csv')

twitter_data = pd.concat([parsed, racism, sexism]).dropna()
twitter_data.head()

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data['Text'], twitter_data['oh_label'], test_size=0.20, random_state=42)

X_train.head()

9327     There is such a diff between reality &amp; wha...
14633    Katie's a fatty!! Model!!!! Hahahaha #MKR #kil...
4197     @Nibelsnarfabarf @srhbutts @GRIMACHU it is rea...
3534     @MaxOfS2D @StephenAtWar Origin is a flaming pi...
4500     No, you don't. @Shut_Up_Jeff: I thought of a r...
Name: Text, dtype: object

In [5]:
y_train.head()

9327     0.0
14633    1.0
4197     0.0
3534     0.0
4500     1.0
Name: oh_label, dtype: float64

In [6]:
vec = TfidfVectorizer(max_features=500)
X_vectrain = vec.fit_transform(X_train).toarray()
X_vectest = vec.transform(X_test).toarray()

In [7]:
### Baseline accuracy, predicting all of one class.
1 - np.mean(y_test)

0.7609513274336284

### Logistic Regression

In [8]:
### Logistic Regression
clf = LogisticRegression(random_state=0).fit(X_vectrain, y_train)
pred = clf.predict(X_vectest)
print("f1_score = ", metrics.f1_score(y_test, pred, average="weighted"))
print("accuracy = ", metrics.accuracy_score(y_test, pred))

f1_score =  0.8433159753737457
accuracy =  0.8535398230088496


/home/jack_wenquwang/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Neural Network

In [9]:
model = Sequential()
model.add(Dense(64,input_shape=(500,)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                32064     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6

In [10]:
model.fit(X_vectrain,y_train,batch_size=32,epochs=10,verbose=1)

Epoch 1/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.3992 - acc: 0.8302
Epoch 2/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.3240 - acc: 0.8704
Epoch 3/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.2776 - acc: 0.8956
Epoch 4/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.2303 - acc: 0.9176
Epoch 5/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.1908 - acc: 0.9332
Epoch 6/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.1584 - acc: 0.9462
Epoch 7/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.1374 - acc: 0.9523
Epoch 8/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.1212 - acc: 0.9573
Epoch 9/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.1083 - acc: 0.9625
Epoch 10/10
1130/1130 [==============================] - 1s 1ms/step - loss: 0.0972 - acc: 0.9662


In [11]:
pred = model.predict_classes(X_vectest)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [12]:
print("f1_score = ", metrics.f1_score(y_test, pred, average="weighted"))
print("accuracy = ", metrics.accuracy_score(y_test, pred))

f1_score =  0.9525810860987441
accuracy =  0.9535398230088495


### CNN

In [13]:
model = Sequential()
embedding_dim = 5
model.add(layers.Embedding(500, embedding_dim, input_length=500))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 5)            2500      
_________________________________________________________________
conv1d (Conv1D)              (None, 496, 128)          3328      
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 7,129
Trainable params: 7,129
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(X_vectrain,y_train,batch_size=32,epochs=10,verbose=1)

Epoch 1/10
1130/1130 [==============================] - 35s 31ms/step - loss: 0.5496 - acc: 0.7636
Epoch 2/10
1130/1130 [==============================] - 35s 31ms/step - loss: 0.5450 - acc: 0.7640
Epoch 3/10
1130/1130 [==============================] - 35s 31ms/step - loss: 0.5447 - acc: 0.7640
Epoch 4/10
1130/1130 [==============================] - 34s 30ms/step - loss: 0.5446 - acc: 0.7640
Epoch 5/10
1130/1130 [==============================] - 34s 30ms/step - loss: 0.5446 - acc: 0.7640
Epoch 6/10
1130/1130 [==============================] - 34s 30ms/step - loss: 0.5449 - acc: 0.7640
Epoch 7/10
1130/1130 [==============================] - 34s 30ms/step - loss: 0.5446 - acc: 0.7640
Epoch 8/10
1130/1130 [==============================] - 35s 31ms/step - loss: 0.5447 - acc: 0.7640
Epoch 9/10
1130/1130 [==============================] - 34s 31ms/step - loss: 0.5447 - acc: 0.7640
Epoch 10/10
1130/1130 [==============================] - 35s 31ms/step - loss: 0.5448 - acc: 0.7640


In [15]:
pred = model.predict_classes(X_vectest)
print("f1_score = ", metrics.f1_score(y_test, pred, average="weighted"))
print("accuracy = ", metrics.accuracy_score(y_test, pred))

f1_score =  0.6576523878906878
accuracy =  0.7609513274336284


### Naive Bayes

In [16]:
model = naive_bayes.MultinomialNB()
model.fit(X_vectrain,y_train)

MultinomialNB()

In [17]:
pred = model.predict(X_vectest)
print("f1_score = ", metrics.f1_score(y_test, pred, average="weighted"))
print("accuracy = ", metrics.accuracy_score(y_test, pred))

f1_score =  0.8001197859908526
accuracy =  0.8279867256637168


### SVM

In [18]:
model = svm.LinearSVC()
model.fit(X_vectrain,y_train)

LinearSVC()

In [19]:
pred = model.predict(X_vectest)
print("f1_score = ", metrics.f1_score(y_test, pred, average="weighted"))
print("accuracy = ", metrics.accuracy_score(y_test, pred))

f1_score =  0.8453593891176628
accuracy =  0.8553097345132743
